## Disable autoscroll

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

## Import libraries

In [13]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as ipw
from IPython.display import display, clear_output

# import sys
# sys.path.append('/home/jovyan/work/aiida-openbis/')
from aiida_openbis.utils import bisutils, gui_utils

import pages_architectures


## Initialise voila package (open in web browser)

In [3]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable voila --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling: voila
- Writing config: /opt/conda/etc/jupyter
    - Validating...
      voila 0.4.3 OK


## Connect to openBIS

In [4]:
session = bisutils.log_in(bisurl="openbis", bisuser="admin", bispasswd="changeit")

In [14]:
main_page.children

(HTML(value='<h1>OpenBIS GUI</h1><hr>'),

## Interface pages

In [15]:
# Main menu page

main_page_elements = []

main_page_title = ipw.HTML(value="<h1>OpenBIS GUI</h1><hr>")

button_new_space_page = ipw.Button(
    description = 'New Space',
    tooltip = 'New Space',
    style = {'description_width': 'initial'}
)

button_new_project_page = ipw.Button(
    description = 'New Project',
    tooltip = 'New Project',
    style = {'description_width': 'initial'}
)

buttons_main_page = ipw.HBox([button_new_space_page, button_new_project_page])

main_page_elements = [main_page_title, buttons_main_page]

main_page = ipw.VBox(main_page_elements)

main_page.children

# New space page

new_space_page_title = ipw.HTML(value="<h1>Create a new space</h1><hr>")

new_space_name_textbox = ipw.Text(
    description = 'New space name:',
    disabled = False,
    layout = ipw.Layout(width='50%', height='80px'),
    style = {'description_width': 'initial'}
)

new_space_description_textbox = ipw.Textarea(
    description = 'New space description:',
    disabled = False,
    layout = ipw.Layout(width='50%', height='80px'),
    style = {'description_width': 'initial'}
)

button_new_space = ipw.Button(
    description = 'Send to openBIS',
    tooltip = 'Send to openBIS',
    style = {'description_width': 'initial'}
)

button_back_main = ipw.Button(
    description = 'Back to Main Menu',
    tooltip = 'Back to Main Menu',
    style = {'description_width': 'initial'}
)

output_new_space_button = ipw.Output()

hbox_new_space_buttons = ipw.HBox([button_new_space, button_back_main])

vbox_new_space_output_buttons = ipw.VBox([output_new_space_button, hbox_new_space_buttons])

new_space_page_elements = [new_space_page_title, new_space_name_textbox,
                           new_space_description_textbox, vbox_new_space_output_buttons]

new_space_page = ipw.VBox(new_space_page_elements)

# New project page

new_project_page_title = ipw.HTML(value="<h1>Create a new project</h1><hr>")

new_project_name_textbox = ipw.Text(
    description = 'New project name:',
    disabled = False,
    layout = ipw.Layout(width='50%', height='80px'),
    style = {'description_width': 'initial'}
)

all_spaces_openbis = list(session.get_spaces().df.code)

new_project_space_dropdownbox = ipw.Dropdown(
    options = all_spaces_openbis,
    disabled = False,
    value = all_spaces_openbis[0],
    description = 'Select a space:',
    style = {'description_width': 'initial'}
)

new_project_description_textbox = ipw.Textarea(
    description = 'New project description:',
    disabled = False,
    layout = ipw.Layout(width='50%', height='80px'),
    style = {'description_width': 'initial'}
)

button_new_project = ipw.Button(
    description = 'Send to openBIS',
    tooltip = 'Send to openBIS',
    style = {'description_width': 'initial'}
)

output_new_project_button = ipw.Output()

hbox_new_project_buttons = ipw.HBox([button_new_project, button_back_main])

vbox_new_project_output_buttons = ipw.VBox([output_new_project_button, hbox_new_project_buttons])

new_project_page_elements = [new_project_page_title, new_project_space_dropdownbox,
                             new_project_name_textbox, new_project_description_textbox,
                             vbox_new_project_output_buttons]

new_project_page = ipw.VBox(new_project_page_elements)

# Buttons' functions

def on_button_new_space_clicked(button):
    # Convert the name to a code that openBIS API can understand
    new_space_code = gui_utils.convert_name_to_code(new_space_name_textbox.value)

    # Check if the space it is being created already exists in the DB
    new_space_not_exists = session.get_spaces(code = new_space_code).df.empty

    # Load values to dictionary
    values = {'description': new_space_description_textbox.value}

    with output_new_space_button:
        clear_output()
        if new_space_not_exists:
            session.new_space(code = new_space_code, **values).save()
            print(f"\nThe space {new_space_name_textbox.value} was successfully created!\n")
        else:
            print(f"\nThe space {new_space_name_textbox.value} already exists!\n")
    
    new_space_name_textbox.value = ''
    new_space_description_textbox.value = ''

# Create a new project

def on_button_new_project_clicked(event):
    # Convert the name to a code that openBIS API can understand
    new_project_code = gui_utils.convert_name_to_code(new_project_name_textbox.value)

    # Check if the space it is being created already exists in the DB
    new_project_not_exists = session.get_projects(code = new_project_code).df.empty

    # Load values to dictionary
    values = {'space': new_project_space_dropdownbox.value,
                'description': new_space_description_textbox.value}

    with output_new_project_button:
        clear_output()
        if new_project_not_exists:
            session.new_project(code = new_project_code, **values).save()
            print(f"\nThe project {new_project_name_textbox.value} was successfully created!\n")
        else:
            print(f"\nThe project {new_project_name_textbox.value} already exists!\n")
    
    new_project_space_dropdownbox.value = all_spaces_openbis[0]
    new_project_name_textbox.value = ''
    new_project_description_textbox.value = ''

def on_button_back_main_clicked(event):
    output_new_space_button.clear_output()
    output_new_project_button.clear_output()
    clear_output()
    display(main_page)

def on_button_new_space_page_clicked(event):
    clear_output()
    display(new_space_page)

def on_button_new_project_page_clicked(event):
    clear_output()
    display(new_project_page)

# Assign functions to the different buttons
button_new_space.on_click(on_button_new_space_clicked)
button_back_main.on_click(on_button_back_main_clicked)
button_new_project.on_click(on_button_new_project_clicked)
button_new_space_page.on_click(on_button_new_space_page_clicked)
button_new_project_page.on_click(on_button_new_project_page_clicked)

# Display interface
display(main_page)

In [ ]:
# Get Person object
person_object = session.get_sample('/PEOPLE/RESEARCHERS/PERS17')

person_id = person_object.code
person_first_name = person_object.props['first-name']
person_last_name = person_object.props['last-name']
person_short_name = person_object.props['short-name']
person_status = person_object.props['status']
person_email = person_object.props['email']
person_work_phone = person_object.props['work-phone']
person_mobile_phone = person_object.props['mobile-phone']
person_office = person_object.props['office-number']

person_parents = person_object.parents
group_object = session.get_sample(person_parents[0])
person_group = group_object.props.name

# person_dataset_object = person_object.get_datasets()[0]
# person_dataset_object.download()

In [ ]:
text_0 = ipw.HTML(value="<h1>Person</h1>")
line_0 = ipw.HTML('<hr>')

file = open("image.png", "rb")
image = file.read()

image_headline = ipw.Image(
    value = image,
    format = 'png',
    width = 300
)

text_1 = ipw.HTML(value="<h2>General Info</h2>")
line_1 = ipw.HTML('<hr>')

# ID
person_id_textbox = ipw.Text(
    value = person_id,
    description = 'ID',
    disabled = True
)

# First Name
person_first_name_textbox = ipw.Text(
    value = person_first_name,
    description = 'First Name',
    disabled = False
)

# Last Name
person_last_name_textbox = ipw.Text(
    value = person_last_name,
    description = 'Last Name',
    disabled = False
)


# Short Name
person_short_name_textbox = ipw.Text(
    value = person_short_name,
    description = 'Short Name',
    disabled = False
)

# Group
person_group_textbox = ipw.Text(
    value = person_group,
    description = 'Group',
    disabled = False
)

# Status
person_status_textbox = ipw.Text(
    value = person_status,
    description = 'Status',
    disabled = False
)

text_2 = ipw.HTML(value="<h2>Contact</h2>")
line_2 = ipw.HTML('<hr>')

# Email
person_email_textbox = ipw.Text(
    value = person_email,
    description = 'Email',
    disabled = False
)

# Work Phone
person_work_phone_textbox = ipw.Text(
    value = person_work_phone,
    description = 'Work Phone',
    disabled = False
)

# Mobile Phone
person_mobile_phone_textbox = ipw.Text(
    value = person_mobile_phone,
    description = 'Mobile Phone',
    disabled = False
)

# Office
person_office_textbox = ipw.Text(
    value = person_office,
    description = 'Office',
    disabled = False
)

In [ ]:
vbox_total = ipw.VBox([text_0,
                       line_0,
                       image_headline, 
                       text_1, 
                       line_1,
                       person_id_textbox, 
                       person_first_name_textbox, 
                       person_last_name_textbox,
                       person_short_name_textbox,
                       person_group_textbox,
                       person_status_textbox,
                       text_2,
                       line_2,
                       person_email_textbox,
                       person_work_phone_textbox,
                       person_mobile_phone_textbox,
                       person_office_textbox])